### Setup

#### TODOS

simspleserial-rsa.c -> input ändern
simpleserial-sra-xmega.c -> load key ändern

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-rsa
#make clean PLATFORM=$1 CRYPTO_TARGET=$2
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CWLITEXMEGA.hex'

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
import numpy as np
import time 
import matplotlib.pylab as plt
import matplotlib

In [ ]:
e = 65537
n = 3231080904348211197507744225057151739674198701955287013863366120571536075433289354791858611634080222605797512592173476389538288555005341834233505795734023

### find numbers

In [ ]:
from sympy import nextprime

# Start with a random 31-byte number and search for a 32-byte prime
start_number = int('F' * 62, 16)  # A large number close to the 32-byte maximum
prime_32_byte = nextprime(start_number)

print(prime_32_byte)

In [ ]:
# generate 8 byte numbers and get next prime number
import random
from sympy import nextprime

#unsigned_8_byte = random.randint(0, 2**64 - 1)
unsigned_8_byte = 6353019252147394183;
print(unsigned_8_byte)



#print(rand)
prime1 = nextprime(unsigned_8_byte)
print("p1:", prime1)
print(prime1.to_bytes(8,'big'))

r = 14091828191315287468

prime2 = nextprime(r)
print('p2:', prime2)
print(prime2.to_bytes(8,'big'))

# p2 = 14091828191315287577


In [ ]:
# prime numbers with 32 bytes 

p = 56842597621398436557813564258526951265526554589515564527115452658451565454963
pb = p.to_bytes(32,'big')
print('p', p, pb, hex(p))
q = 56842597621398436557813564258526951265526554589515564527115452658451565454621
qb = q.to_bytes(32,'big')
print('q', q, qb, hex(q))

N = p*q
#print(N) # 3231080904348211197507744225057151739674198701955287013863366120571536075433289354791858611634080222605797512592173476389538288555005341834233505795734023
Nb = N.to_bytes(64,'big')
print('N', N, Nb, hex(N))

e = 65537
eb = e.to_bytes(32,'big')
print('e', e, eb, hex(e))

euler = (p-1)*(q-1)
#print(euler)
# 3231080904348211197507744225057151739674198701955287013863366120571536075433175669596615814760964595477280458689642423280359257425951110928916602664824440

#imversis zu e mod euler: e*d + k * euler = 1 = ggT(e, euler)
# www.ardt-brunner.de : 
# -1378424264227407981310084697016538832866181264140376727964445326534012498787496506496357501333625419609522626066248264529892495512033921455995533727601607
# * e +
# 27959
# * euler = 1 



In [ ]:
d_ = -1378424264227407981310084697016538832866181264140376727964445326534012498787496506496357501333625419609522626066248264529892495512033921455995533727601607


d = d_ + euler
# print(d)
# 1852656640120803216197659528040612906808017437814910285898920794037523576645679163100258313427339175867757832623394158750466761913917189472921068937222833
db = d.to_bytes(64,'big')
print('d', d, db, hex(d))

#test
#t = (d * e ) % euler
# print(t)

dp = d % (p-1)
#print(dp)
#42226251814971434209944815538190405761053455455552821444414846768645413494283
dpb = dp.to_bytes(32,'big')
print('dp', dp, dpb, hex(dp))

dq = d % (q-1)
#print(dq)
#47798018101494521266834730193984493603489676013146974160013511154837836956813
dqb = dq.to_bytes(32,'big')
print('dq', dq, dqb, hex(dq))


qinv = pow(q, -1, p)
#print(qinv)
#21773041778956711079162505607798335133871282605925552494304457012447821855556
qinvb = qinv.to_bytes(32,'big')
print('qinv', qinv, qinvb, hex(qinv))

#test
#t2 =(qinv * q) % p
#print(t2)

### capture trace setup

In [ ]:
scope.clock.adc_src = "clkgen_x1"
#scope.adc.samples = 1000
scope.adc.samples = 100

#scope.adc.timeout = 60

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    scope.arm()
    target.simpleserial_write('h', text)

    # get debug output
    rec_str = target.read(3000, timeout=40000)
    received_str = rec_str.replace("\\r\\n", "\n")
    print(received_str)
    
    ret = scope.capture()
    if ret:
        return None
    target.simpleserial_wait_ack()
    return scope.get_last_trace()

### test numbers for 2 Byte prim numbers

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 1
c = (msg ** e) % n

msg2_d1 = c.to_bytes(64,'big')

print(c, msg2_d1, type(msg2_d1))

trace2_d1 = capture_RSA_trace(scope, target, msg2_d1)
plt.figure()
plt.plot(trace2_d1, 'r')

In [ ]:
%matplotlib ipympl


msg = 62558
c = (msg ** e) % n

msg2 = c.to_bytes(64,'big')

print(c, msg2, hex(c))

trace2 = capture_RSA_trace(scope, target, msg2)
plt.figure()
plt.plot(trace2, 'r')

### Key Reconstuction

* find plaintexts x_1 and x_2 mit diff(x_1) = 0 und diff(x_2) = 1.
* mittels binäre suche: finde x mit diff(x-1) = 0 und diff(x) = 1 -> x ist Vielfaches von p 
* ggt(x, n) = p
* x^e mod n

### Algorithmus

Reconstruction of p when its bitlength is half bitlength of n
INPUT: modulus n with the bitlength t, public exponent e.
OUTPUT: prime p such that p divides n.

t = bitlenght von n 
p,q haben bitlänge t/2

x = 2 ** (t/2)
m = x/2 

werte liegen bei 2 und bei 12 -> mitte  wählen bei mit debug 
werte liegen bei 2 und bei 11 -> mitte ist 7/6

In [ ]:
# 32Byte Primzahlen 32*8=512 

t = 512

# x= 2 ** (t/2) 
# m = x/2 
# l = 0 

x = 2**256 #upper bound
m = x//2 #middle
l = 0 # lower bound

print(x, m, l)

In [ ]:

 
plt.figure()

ref_trace_msg_ = 1 # loop wird nicht ausgeführt, diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(64,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    #enc = (m ** e) % n
    
    print(m, type(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(64,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print("{:1} diff = {:2}".format(m, diff))
    
    #check if diff(m) = 1
    # debug: sad von d1 und d0(ref) = 6  und sad von d0 und d0(ref) = 1-> wähle diff 3 //getestet mit :5
    # no debug sad von d1 und d0 = 6 und sad von d0 und d0 = 3,5 -> wähle diff 5/6
    if diff < 5: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)
    

In [ ]:
#get new values from first try
#upper bound: values that was bigger than the last m
x = 56842597621398436557813564261652795744130418239689185364786678426454227681280
#last values
m = 56842597621398436557813564258729792469468612403281815699354112387142362595328
#values that was smaller that the last m 
l = 56842597621398436557813564255806789194806806566874446033921546347830497509376
 
plt.figure()

ref_trace_msg_ = 1 # loop wird nicht ausgeführt, diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(64,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    #enc = (m ** e) % n
    
    print(m, type(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(64,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print("{:1} diff = {:2}".format(m, diff))
    
    #check if diff(m) = 1
    # debug: sad von d1 und d0(ref) = 6  und sad von d0 und d0(ref) = 1-> wähle diff 3 //getestet mit :5
    # no debug sad von d1 und d0 = 7 und sad von d0 und d0 = 2 -> wähle diff 5/6
    if diff < 5: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)
    

In [ ]:
#get new values from first try
#upper bound: values that was bigger than the last m
x = 56842597621398436557813564258526951337451936805444658656114666078300962553856
#last values
m = 56842597621398436557813564258526951293895793839564535332802716327034631487488
#values that was smaller that the last m 
l = 56842597621398436557813564258526951250339650873684412009490766575768300421120
 
plt.figure()

ref_trace_msg_ = 1 # loop wird nicht ausgeführt, diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(64,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    #enc = (m ** e) % n
    
    print(m, type(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(64,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print("{:1} diff = {:2}".format(m, diff))
    
    #check if diff(m) = 1
    # debug: sad von d1 und d0(ref) = 6  und sad von d0 und d0(ref) = 1-> wähle diff 3 //getestet mit :5
    # no debug sad von d1 und d0 = 6,5 und sad von d0 und d0 = 3,5 -> wähle diff 5/6
    if diff < 5: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)

#error 
#(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

#5684259762139843655781356425852695126552655458951556452711545265 1275557011456 diff = 3.296875
#5684259762139843655781356425852695126552655458951556452711545265 5673603522560 <class 'int'> 




#(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

#5684259762139843655781356425852695126552655458951556452711545265 5673603522560 diff = 7.171875
#5684259762139843655781356425852695126552655458951556452711545265 3474580267008 <class 'int'> 

#p =
#5684259762139843655781356425852695126552655458951556452711545265 8451565454963


In [ ]:
#get new values from first try
#upper bound: values that was bigger than the last m
x = 56842597621398436557813564258526951265526554589515564527115452660071650033664
#last values
m = 56842597621398436557813564258526951265526554589515564527115452655673603522560
#values that was smaller that the last m 
l = 56842597621398436557813564258526951265526554589515564527115452651275557011456
 
plt.figure()

ref_trace_msg_ = 1 # loop wird nicht ausgeführt, diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(64,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    #enc = (m ** e) % n
    
    print(m, type(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(64,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print("{:1} diff = {:2}".format(m, diff))
    
    #check if diff(m) = 1
    # debug: sad von d1 und d0(ref) = 6  und sad von d0 und d0(ref) = 1-> wähle diff 3 //getestet mit :5
    # no debug sad von d1 und d0 = 7 und sad von d0 und d0 = 3,5 -> wähle diff 5/6
    if diff < 5: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)



In [ ]:
#get new values from first try
#upper bound: values that was bigger than the last m
x = 56842597621398436557813564258526951265526554589515564527115452658451565477888
#last values
m = 56842597621398436557813564258526951265526554589515564527115452658451565461504
#values that was smaller that the last m 
l = 56842597621398436557813564258526951265526554589515564527115452658451565445120
 
plt.figure()

ref_trace_msg_ = 1 # loop wird nicht ausgeführt, diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(64,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    #enc = (m ** e) % n
    
    print(m, type(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(64,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print("{:1} diff = {:2}".format(m, diff))
    
    #check if diff(m) = 1
    # debug: sad von d1 und d0(ref) = 6  und sad von d0 und d0(ref) = 1-> wähle diff 3 //getestet mit :5
    # no debug sad von d1 und d0 = 7 und sad von d0 und d0 = 3,5 -> wähle diff 5/6
    if diff < 5: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)



p = 56842597621398436557813564258526951265526554589515564527115452658451565454 963

q = 56842597621398436557813564258526951265526554589515564527115452658451565454 621


q = N // p
euler = (p-1)*(q-1)

-> d durch erweiterter euklidischer algo 

In [ ]:
scope.dis()
target.dis()